In [ ]:
!pip install openpyxl

In [ ]:
!pip install pandas 

In [ ]:
!pip install pymongo

In [5]:
import pandas as pd 
from pymongo import MongoClient

In [48]:
url = "mongodb://localhost:27017/"
conn = MongoClient(url)
db = conn['zameen']
print('Data connection is successful')

Data connection is successful


In [49]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'zameen')

In [50]:
query = {
#     'city' : 'Islamabad',
#     'area' : '5 Marla'
}

df = pd.DataFrame(list(db.plots_data.find()))

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df['price'].str[:-4].head()

In [ ]:
df['city'].value_counts()

In [ ]:
df.pivot_table(index='id', columns='city', values='area', aggfunc='sum')

In [ ]:
df.iloc[0]

In [ ]:
df.pivot_table(index='area', columns='city', values='price', aggfunc='first')

In [ ]:
df[
    (df['area'] == '1 Kanal')
][['city','area', 'price']].head()

In [ ]:
df['area'].value_counts()

In [ ]:
df.describe()

In [ ]:
df.price.dtype

In [ ]:
# df[df['price'].str.contains('Crore')][['city', 'price']].head()
df['price'].head()

In [ ]:
# Convert price column to one unit e.g crore or lakh or 1000 
# cast column to integer

# Convert Area column to one unit e.g crore or lakh or 1000 
# cast column to integer

In [ ]:
# bar graph of cities on x axis
# y axis shows count of 5 marla, 10 marla, 20 marla and so on


# Min, max and average price range for all cities

# show correlation between price and area
# show corelation between price and city



In [ ]:
def convert_price_to_numeric(price_str):
    if 'Lakh' in price_str:
        return float(price_str.replace(' Lakh', ''))
    
    elif 'Crore' in price_str:
        return float(price_str.replace(' Crore', '')) * 100
    
    elif 'Arab' in price_str:
        return float(price_str.replace(' Arab', '')) * 10000
    
    else:
        return price_str

# Apply the conversion function to the 'price' column
df['price_in_numeric'] = df['price'].apply(convert_price_to_numeric)


In [ ]:
df['Plot Price (in Lakh)'] = df['price_in_numeric'].map(int)

In [ ]:
df[['price', 'Plot Price (in Lakh)']].head()

In [ ]:
df[df['price'].str.contains('Arab')]['price']

In [ ]:
# df['price'].value_counts()

In [ ]:
df['area'].value_counts().head()

In [ ]:
df['Plot_str'] = df['area'].str.split().str.get(1)

In [ ]:
df['Plot_str'].value_counts()

In [ ]:
# df[df['area'].str.contains('Kanal')]

In [ ]:
# df['area'].str.split().str.get(0)

In [ ]:
df['area'].head()

In [ ]:
def convert_to_kanals(area):
    area = area.replace(',', '')  # Remove comma from the area string
    
    if 'Kanal' in area:
        return float(area.split()[0]) 
    
    elif 'Marla' in area:
        return float(area.split()[0]) / 20
    
    elif 'Sq. Yd.' in area:
        return float(area.split()[0]) / 20
    
    else:
        return None

In [ ]:
df['area_in_kanals'] = df['area'].apply(convert_to_kanals)

In [ ]:
# to convert square yard to marla i.e sq_yd/20.004
def square_yards_to_marla(square_yards):
    return square_yards / 20.0004

square_yards_to_marla(1000)

In [ ]:
def convert_to_marla(area):
    area = area.replace(',', '')  # Remove comma from the area string
    
    if 'Kanal' in area:
        return float(area.split()[0]) * 20
    
    elif 'Marla' in area:
        return float(area.split()[0])
    
    elif 'Sq. Yd.' in area:
        return float(area.split()[0]) / 20
    
    else:
        return None

In [ ]:
df['area_in_marla'] = df['area'].apply(convert_to_marla)

In [ ]:
df[['area', 'area_in_kanals', 'area_in_marla']].head()

In [ ]:
df[
    df['area'].str.contains('Kanal')
][['area', 'area_in_kanals', 'area_in_marla']].head()

In [ ]:
# df[df['area'].str.contains(',')]['area']

In [ ]:
mini_df = df[["city","area_in_marla","Plot Price (in Lakh)"]]

In [ ]:
df.pivot_table(index="city", columns="area_in_marla", aggfunc="count")

In [ ]:
df[df['area_in_kanals']==1250.0][['area', 'area_in_kanals']]

In [ ]:
df

In [ ]:
correlation = df['price_in_numeric'].corr(df['area_in_kanals'])

print("Correlation between 'price_in_numeric' and 'area_in_kanals':", correlation)

In [ ]:
df.columns

In [ ]:
df = df[['city', 'location', 'area_in_marla', 'Plot Price (in Lakh)', ]]

In [ ]:
data = 'plots_data_update.xlsx'
df.to_excel(data)

In [ ]:
!pip install xlrd==1.2.0

In [2]:
some_df = pd.read_excel('plots_data_update.xlsx', )

In [ ]:
!pip install xlrd

In [8]:
some_df['bucket_1'] = pd.qcut(some_df['Plot Price (in Lakh)'], q=10)

In [10]:
some_df['bucket_2'] = pd.qcut(some_df['Plot Price (in Lakh)'], q=10, precision=0)

In [43]:
some_df['marla_bucket'] = pd.qcut(some_df['area_in_marla'], q=4, precision=0)

In [40]:
some_df.marla_bucket.value_counts()

(10.0, 20.0]      455
(2.0, 6.0]        428
(6.0, 10.0]       297
(20.0, 1250.0]    250
Name: marla_bucket, dtype: int64

In [44]:
some_df[['area_in_marla', 'marla_bucket']].head()

,area_in_marla,marla_bucket
0,10.0,"(6.0, 10.0]"
1,5.0,"(2.0, 6.0]"
2,5.0,"(2.0, 6.0]"
3,10.0,"(6.0, 10.0]"
4,10.0,"(6.0, 10.0]"


In [13]:
some_df['bucket_1'].value_counts()

(125.0, 170.0]      148
(51.8, 70.0]        147
(2.999, 36.0]       146
(570.5, 13000.0]    143
(347.6, 570.5]      143
(170.0, 240.0]      143
(94.0, 125.0]       143
(70.0, 94.0]        140
(36.0, 51.8]        140
(240.0, 347.6]      137
Name: bucket_1, dtype: int64

In [28]:
some_df.head(2)

,Unnamed: 0,city,location,area_in_marla,Plot Price (in Lakh),bucket_1,bucket_2,marla_bucket
0,0,Lahore,"Bahria Orchard Phase 4 - Block G5, Bahria Orch...",10.0,54,"(51.8, 70.0]","(52.0, 70.0]","(2.999, 1250.0]"
1,1,Lahore,"Park View City - Crystal Block, Park View City",5.0,60,"(51.8, 70.0]","(52.0, 70.0]","(2.999, 1250.0]"


In [22]:
some_df.bucket_1.unique()

[(51.8, 70.0], (36.0, 51.8], (125.0, 170.0], (94.0, 125.0], (570.5, 13000.0], (70.0, 94.0], (240.0, 347.6], (170.0, 240.0], (347.6, 570.5], (2.999, 36.0]]
Categories (10, interval[float64]): [(2.999, 36.0] < (36.0, 51.8] < (51.8, 70.0] < (70.0, 94.0] ... (170.0, 240.0] < (240.0, 347.6] < (347.6, 570.5] < (570.5, 13000.0]]

In [23]:
some_df.to_excel("spreadsheets/working.xlsx", index=False)

In [54]:
rental_plot_df =pd.DataFrame(list(db.rental_plots_data.find()))                 

In [55]:
rental_plot_df.head()

,_id,area,city,location,price,property_link,property_title
0,64baa8941469c4a520d0b420,6 Marla,Lahore,"Grand Avenue Housing Scheme - Block A, Grand A...",20 Lakh,/Property/grand_avenues_housing_scheme_grand_a...,feroz pur road grand avenue
1,64baa8941469c4a520d0b421,5 Marla,Lahore,"Bahria Town - Tauheed Extension Block, Bahria ...",48 Lakh,/Property/bahria_town_sector_f_bahria_town_tau...,5 MARLA POSSESSED PLOT FOR SALE TAUHEED EXTENS...
2,64baa8941469c4a520d0b422,2 Kanal,Lahore,"DHA Phase 5 - Block F, DHA Phase 5",9 Crore,/Property/dha_phase_5_dha_phase_5_block_f_main...,Main Boulevard Plot Is Available At Azeem Estate
3,64baa8941469c4a520d0b423,3 Marla,Lahore,"Etihad Town Phase 1, Etihad Town",15 Lakh,/Property/etihad_town_etihad_town_phase_1_new_...,NEW DEAL 3 MARLA PLOT AVAILABLE ON EASY INSTAL...
4,64baa8941469c4a520d0b424,5 Marla,Lahore,"Bankers Avenue Cooperative Housing Society, La...",50 Thousand,/Property/lahore_bankers_avenue_cooperative_ho...,5 marla


In [57]:
rental_plot_df[['area', 'price', 'city']].tail()

,area,price,city
60,183 Sq. Yd.,55 Thousand,Karachi_Clifton
61,167 Sq. Yd.,50 Thousand,Karachi_Clifton
62,233 Sq. Yd.,1.1 Lakh,Karachi_Clifton
63,500 Sq. Yd.,1.5 Lakh,Karachi_Clifton
64,"1,000 Sq. Yd.",15 Crore,Karachi_Clifton
